# Contract Clause Classification System using DSPy

This notebook implements an advanced contract clause classification system using DSPy and Azure OpenAI. The system employs chain-of-thought reasoning and zero-shot learning to identify specific clauses in legal contracts.

## Zero-Shot Learning with Chain-of-Thought
This implementation uses:
1. Chain-of-thought reasoning for detailed analysis
2. Zero-shot learning without requiring training examples
3. Legal-specific prompting with native DSPy prompt handling
4. Optimized classification pipeline

## Setup and Configuration
First, we'll set up our environment and initialize DSPy with Azure OpenAI.

In [2]:
import os 
import dspy
import pandas as pd
from dotenv import dotenv_values, load_dotenv
from openai import AzureOpenAI

# Load environment variables
load_dotenv()

# Azure OpenAI Configuration
azure_endpoint = os.getenv("AZURE_OPENAI_API_EASTUS_ENDPOINT")
api_key = os.getenv("AZURE_OPENAI_EASTUS_API_KEY")
deployment = 'gpt-4o-mini-eastus-0718'

# Initialize DSPy with Azure OpenAI
turbo = dspy.AzureOpenAI(
    api_key=api_key,
    api_version="2024-06-01",
    api_base=azure_endpoint,
    model=deployment
)

# Configure DSPy
dspy.configure(lm=turbo)

/Users/ozgurguler/Developer/Projects/dspy/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Contract Analysis Signatures
We define two main components for contract analysis:
1. ContractAnalyzer: Performs detailed structural analysis
2. ChainOfThoughtClassifier: Conducts step-by-step legal reasoning

class ContractAnalyzer(dspy.Signature):
    context = dspy.InputField(desc="The contract text to analyze")
    analysis = dspy.OutputField(desc="Step-by-step analysis of contract structure")
    conclusion = dspy.OutputField(desc="Final summary of identified sections")

class ChainOfThoughtClassifier(dspy.Signature):
    context = dspy.InputField(desc="The contract text to classify")
    clause_type = dspy.InputField(desc="The type of clause to identify")
    reasoning = dspy.OutputField(desc="Step-by-step legal analysis of the clause presence")
    decision = dspy.OutputField(desc="Final classification (Present/Absent) with justification")

In [3]:
from typing import ClassVar

class ContractAnalyzer(dspy.Signature):
    context = dspy.InputField(desc="The contract text to analyze")
    analysis = dspy.OutputField(desc="Step-by-step analysis of contract structure")
    conclusion = dspy.OutputField(desc="Final summary of identified sections")
    
    prompt: ClassVar[str] = """
{prefix}
{instructions}

Context:
{context}

Please provide a step-by-step analysis of the contract's structure and a final summary of the identified sections.

Analysis: {analysis}
Conclusion: {conclusion}
"""

class ChainOfThoughtClassifier(dspy.Signature):
    context = dspy.InputField(desc="The contract text to classify")
    clause_type = dspy.InputField(desc="The type of clause to identify")
    reasoning = dspy.OutputField(desc="Step-by-step legal analysis of the clause presence")
    decision = dspy.OutputField(desc="Final classification (Present/Absent) with justification")
    
    prompt: ClassVar[str] = """
{prefix}
{instructions}

Context:
{context}

Clause Type:
{clause_type}

Please determine whether the specified clause type is present in the context, providing a step-by-step legal analysis and a final classification.

Reasoning: {reasoning}
Decision: {decision}
"""


## Pipeline Implementation
The pipeline combines analysis and classification using DSPy's Predict module.

class ContractPipeline(dspy.Module):
    def __init__(self):
        super().__init__()
        self.analyzer = dspy.Predict(ContractAnalyzer)
        self.classifier = dspy.Predict(ChainOfThoughtClassifier)
        self.logger = logging.getLogger('ContractPipeline')
    
    def forward(self, contract_text, clause_type):
        # First, analyze contract structure
        # Log the prompt used
        self.logger.info(f"Analyzer Prompt: {self.analyzer.last_prompt}")
        analysis = self.analyzer(context=contract_text)
        
        # Log the prompt used
        self.logger.info(f"Analyzer Prompt: {self.analyzer.last_prompt}")

        # Then perform classification with reasoning
        result = self.classifier(
            context=contract_text,
            clause_type=clause_type
        )
        # Log the prompt used
        self.logger.info(f"Classifier Prompt: {self.classifier.last_prompt}")

        return result.decision, result.reasoning

In [4]:
class ContractPipeline(dspy.Module):
    def __init__(self):
        super().__init__()
        self.analyzer = dspy.Predict(ContractAnalyzer)
        self.classifier = dspy.Predict(ChainOfThoughtClassifier)
        self.logger = logging.getLogger('ContractPipeline')

    def forward(self, contract_text, clause_type):
        # Prepare inputs
        analyzer_input = {
            'context': contract_text
        }
        classifier_input = {
            'context': contract_text,
            'clause_type': clause_type
        }

        # Build prompts
        analyzer_prompt = ContractAnalyzer.prompt.format(
            prefix=dspy.settings.prefix,
            instructions=dspy.settings.instructions,
            **analyzer_input,
            analysis='{analysis}',
            conclusion='{conclusion}'
        )
        classifier_prompt = ChainOfThoughtClassifier.prompt.format(
            prefix=dspy.settings.prefix,
            instructions=dspy.settings.instructions,
            **classifier_input,
            reasoning='{reasoning}',
            decision='{decision}'
        )

        # Log prompts
        self.logger.info(f"Analyzer Prompt:\n{analyzer_prompt}")
        self.logger.info(f"Classifier Prompt:\n{classifier_prompt}")

        # First, analyze contract structure
        analysis = self.analyzer(**analyzer_input)
        
        # Then perform classification with reasoning
        result = self.classifier(**classifier_input)
        
        return result.decision, result.reasoning

## Zero-Shot Optimization
The optimizer enhances the pipeline with legal-specific prompting using DSPy's native prompt handling.

# Updated ZeroShotOptimizer with prompt handling
class ZeroShotOptimizer:
    def __init__(self, model):
        self.model = model
        # Configure logging
        logging.basicConfig(level=logging.INFO)
        self.logger = logging.getLogger('ZeroShotOptimizer')
    
    def optimize(self, pipeline):
        # Log the optimization process
        self.logger.info("Starting optimization...")
        # Directly adjust the prompt context using DSPy settings
        dspy.settings.configure(
            lm=self.model,
            prefix="You are an expert legal analyst.",
            instructions="Analyze the contract's structure and determine the presence of specific clauses with detailed reasoning."
        )
        self.logger.info("Optimization complete.")
        return pipeline

# Define the ContractPipeline with analysis and classification
class ContractPipeline(dspy.Module):
    def __init__(self):
        super().__init__()
        self.analyzer = dspy.Predict(ContractAnalyzer)
        self.classifier = dspy.Predict(ChainOfThoughtClassifier)
    
    def forward(self, contract_text, clause_type):
        # First, analyze contract structure
        analysis = self.analyzer(context=contract_text)
        
        # Then perform classification with reasoning
        result = self.classifier(
            context=contract_text,
            clause_type=clause_type
        )
        
        return result.decision, result.reasoning


In [5]:
class ZeroShotOptimizer:
    def __init__(self, model):
        self.model = model
        self.legal_terms = ['pursuant to', 'hereinafter', 'clause', 'provision', 'agreement']
    
    def _evaluate_reasoning_quality(self, reasoning):
        # Check for legal terminology
        term_presence = sum(1 for term in self.legal_terms if term in reasoning.lower()) / len(self.legal_terms)
        
        # Check analysis depth (number of distinct points made)
        analysis_points = len(reasoning.split('.'))
        
        return 0.5 * term_presence + 0.5 * min(1.0, analysis_points / 5)
    
    def _evaluate_confidence(self, analysis):
        # Evaluate thoroughness of analysis
        sections = len(analysis.split('\n'))
        return min(1.0, sections / 3)
    
    def _evaluate_pipeline(self, pipeline, eval_set):
        total_score = 0
        for example in eval_set:
            pred = pipeline(example['context'], example['clause_type'])
            total_score += self.metric_fn(pred, example['gold'])
        return total_score / len(eval_set)
    
    def optimize(self, pipeline):
        # Create optimizer with multiple metrics
        optimizer = dspy.teleprompt.BasicOptimizer(
            metric="weighted_accuracy",
            num_rounds=5,
            max_iterations=10,
            temp_range=[0.0, 0.7],  # Temperature range to explore
            max_tokens_range=[100, 500]  # Response length range
        )
        
        # Enhanced metric function with semantic similarity
        def metric_fn(pred, gold):
            # Direct match score
            exact_match = pred.decision.lower() == gold.lower()
            
            # Reasoning quality score
            reasoning_score = self._evaluate_reasoning_quality(pred.reasoning)
            
            # Confidence score
            confidence_score = self._evaluate_confidence(pred.analysis)
            
            # Weighted combination of scores
            return 0.6 * exact_match + 0.2 * reasoning_score + 0.2 * confidence_score
        
        # Store metric_fn as instance attribute for _evaluate_pipeline
        self.metric_fn = metric_fn
        
        # Load and prepare validation data
        df = pd.read_csv('contracts_advanced/contract_labels.csv')
        df.columns = [col.lower() for col in df.columns]
        
        # Create comprehensive validation set
        valset = []
        for _, row in df.iterrows():
            with open(row['filename'], 'r') as f:
                contract_text = f.read()
                
            for clause, column in {
                "Non-Disclosure Agreement (NDA) clause": "contains_nda",
                "Termination Clause": "contains_termination",
                "Indemnity Clause": "contains_indemnity",
                "Force Majeure Clause": "contains_force_majeure",
                "Data Protection Clause": "contains_data_protection"
            }.items():
                if row[column] != "Unknown":
                    valset.append({
                        'context': contract_text,
                        'clause_type': clause,
                        'gold': row[column]
                    })
        
        # Implement k-fold cross-validation
        k = 5
        fold_size = len(valset) // k
        best_pipeline = None
        best_score = 0
        
        for i in range(k):
            # Split validation set
            val_fold = valset[i * fold_size:(i + 1) * fold_size]
            
            # Optimize pipeline for this fold
            optimized = optimizer.optimize(
                module=pipeline,
                trainset=[],  # Zero-shot learning
                valset=val_fold,
                metric=metric_fn
            )
            
            # Evaluate on remaining data
            score = self._evaluate_pipeline(optimized, valset[:i * fold_size] + valset[(i + 1) * fold_size:])
            
            if score > best_score:
                best_score = score
                best_pipeline = optimized
        
        return best_pipeline

## Example Usage
Let's test the pipeline with sample contracts from our dataset.

In [6]:
# Create and configure the pipeline
pipeline = ContractPipeline()
optimizer = ZeroShotOptimizer(turbo)
optimized_pipeline = optimizer.optimize(pipeline)

# Load test data
df = pd.read_csv('contracts_advanced/contract_labels.csv')
df.columns = [col.lower() for col in df.columns]

# Test with a sample contract from the dataset
with open(df['filename'].iloc[0], 'r') as f:
    test_contract = f.read()

# Test classification
clause_type = "Non-Disclosure Agreement (NDA) clause"
result = optimized_pipeline(test_contract, clause_type)
print(f"Classification: {result[0]}")
print(f"Reasoning: {result[1]}")

NameError: name 'logging' is not defined

## Model Comparison: GPT-4O-Mini vs GPT-4O
Let's evaluate and compare the performance of both models on our contract classification task.

In [7]:
# Model deployments
deployments = {
    "gpt-4o-mini": "gpt-4o-mini-eastus-0718",
    "gpt-4o": "gpt-4o-eastus-0806"
}

# Clause types
clauses = [
    "Non-Disclosure Agreement (NDA) clause",
    "Termination Clause",
    "Indemnity Clause",
    "Force Majeure Clause",
    "Data Protection Clause"
]

# Clause to column mapping
clause_column_map = {
    "Non-Disclosure Agreement (NDA) clause": "contains_nda",
    "Termination Clause": "contains_termination",
    "Indemnity Clause": "contains_indemnity",
    "Force Majeure Clause": "contains_force_majeure",
    "Data Protection Clause": "contains_data_protection"
}


# Function to evaluate model
def evaluate_model(deployment_name, df, clauses):
    # Configure model
    turbo = dspy.AzureOpenAI(
        api_key=api_key,
        api_version="2024-06-01",
        api_base=azure_endpoint,
        model=deployment_name
    )
    dspy.configure(lm=turbo)
    
    # Initialize and optimize pipeline
    pipeline = ContractPipeline()
    optimizer = ZeroShotOptimizer(turbo)
    optimized_pipeline = optimizer.optimize(pipeline)
    
    # Evaluate each contract
    results = {clause: {"correct": 0, "total": 0} for clause in clauses}
    
    for _, row in df.iterrows():
        with open(row["filename"], "r") as f:
            contract_text = f.read()
            
        for clause in clauses:
            column = clause_column_map[clause]
            
            if row[column] != "Unknown":
                results[clause]["total"] += 1
                prediction, _ = optimized_pipeline(contract_text, clause)
                # Normalize prediction and actual value for comparison
                prediction_normalized = prediction.strip().split('.')[0].lower()
                actual_normalized = row[column].strip().lower()
                if prediction_normalized == actual_normalized:
                    results[clause]["correct"] += 1
    
    # Calculate accuracies
    accuracies = {
        clause: (results[clause]["correct"] / results[clause]["total"] * 100)
        if results[clause]["total"] > 0 else 0
        for clause in clauses
    }
    
    return accuracies

# EVAL

In [8]:
import logging
import pandas as pd
import os
from collections import defaultdict
from tabulate import tabulate

# Load contract labels from CSV
df = pd.read_csv('contracts_advanced/contract_labels.csv')
df.columns = [col.lower() for col in df.columns]

# Clause types for evaluation
clause_types = [
    "Non-Disclosure Agreement (NDA) clause",
    "Termination Clause",
    "Indemnity Clause",
    "Force Majeure Clause",
    "Data Protection Clause"
]

# Clause to column mapping in CSV
clause_column_map = {
    "Non-Disclosure Agreement (NDA) clause": "contains_nda",
    "Termination Clause": "contains_termination",
    "Indemnity Clause": "contains_indemnity",
    "Force Majeure Clause": "contains_force_majeure",
    "Data Protection Clause": "contains_data_protection"
}

# Model deployments
deployments = {
    "gpt-4o-mini": "gpt-4o-mini-eastus-0718",
    "gpt-4o": "gpt-4o-eastus-0806"
}

# Initialize counters for each clause type
accuracy_counts = {model: defaultdict(lambda: {'correct': 0, 'total': 0}) for model in deployments}

# Run evaluation for each model
for model_name, deployment_name in deployments.items():
    print(f"\nEvaluating model: {model_name} ({deployment_name})")

    # Configure DSPy with the current model
    turbo = dspy.AzureOpenAI(
        api_key=api_key,
        api_version="2024-06-01",
        api_base=azure_endpoint,
        model=deployment_name
    )
    dspy.configure(lm=turbo)
    
    # Initialize and optimize the pipeline
    pipeline = ContractPipeline()
    optimizer = ZeroShotOptimizer(turbo)
    optimized_pipeline = optimizer.optimize(pipeline)

    # Process each contract and evaluate every 5 contracts
    contract_count = 0
    for idx, row in df.iterrows():
        contract_path = os.path.join(row["filename"])
        
        # Check if the contract file exists
        if not os.path.exists(contract_path):
            print(f"Contract file {row['filename']} not found, skipping.")
            continue
        
        # Read the contract content
        with open(contract_path, "r") as f:
            contract_text = f.read()
        
        contract_count += 1
        print(f"\nProcessing contract {contract_count}: {row['filename']}")
        
        # Evaluate each clause type in the contract
        for clause_type in clause_types:
            column_name = clause_column_map[clause_type]
            
            # Skip if there's no label for this clause type in the CSV
            if row[column_name] == "Unknown":
                continue
            
            # Run the pipeline for clause classification
            result = optimized_pipeline(contract_text, clause_type)
            classification_decision, reasoning = result
            
            # Normalize the decision for comparison
            predicted = classification_decision.strip().split('.')[0].lower()
            actual = row[column_name].strip().lower()
            
            # Update counts for accuracy tracking
            accuracy_counts[model_name][clause_type]['total'] += 1
            if predicted == actual:
                accuracy_counts[model_name][clause_type]['correct'] += 1
            
            # Output results for each clause
            print(f"\nClause Type: {clause_type}")
            print(f"Expected: {actual}")
            print(f"Predicted: {predicted}")
            print(f"Reasoning:\n{reasoning}\n")
            print("Result:", "Correct" if predicted == actual else "Incorrect", "\n")
        
        # Every 5 contracts, output intermediate accuracy results
        if contract_count % 5 == 0:
            print(f"Intermediate Accuracy after {contract_count} contracts for model {model_name}:")
            intermediate_results = [
                [clause_type,
                 accuracy_counts[model_name][clause_type]['correct'],
                 accuracy_counts[model_name][clause_type]['total'],
                 (accuracy_counts[model_name][clause_type]['correct'] / max(accuracy_counts[model_name][clause_type]['total'], 1)) * 100]
                for clause_type in clause_types
            ]
            print(tabulate(intermediate_results, headers=["Clause Type", "Correct", "Total", "Accuracy (%)"], tablefmt="grid"))
    
    # Final summary table for each model
    print(f"\nFinal Accuracy Summary for model {model_name}:")
    final_results = [
        [clause_type,
         accuracy_counts[model_name][clause_type]['correct'],
         accuracy_counts[model_name][clause_type]['total'],
         (accuracy_counts[model_name][clause_type]['correct'] / max(accuracy_counts[model_name][clause_type]['total'], 1)) * 100]
        for clause_type in clause_types
    ]
    print(tabulate(final_results, headers=["Clause Type", "Correct", "Total", "Accuracy (%)"], tablefmt="grid"))



Evaluating model: gpt-4o-mini (gpt-4o-mini-eastus-0718)


NameError: name 'logging' is not defined